# Plan de limpieza : de Raw a Processsed

Aquí te presento un plan de limpieza completo y estructurado, diseñado para tu proyecto y tu flujo de trabajo (de `raw` a `processed`). Lo dividiremos en:

1.  **La Filosofía y el Flujo de Trabajo.**
2.  **La Checklist de Limpieza (El "Qué" y el "Cómo").**
3.  **Un Script de Ejemplo (`clean_data.py`) que implementa el plan.**

---

### 1. La Filosofía y el Flujo de Trabajo

**El Principio:** Nunca modifiques tus archivos `raw`. Son tu fuente de verdad inmutable. El proceso de limpieza siempre debe ser: **Leer de `/data/raw/` -> Aplicar Limpieza -> Escribir en `/data/processed/`.**

**El Flujo de Trabajo:**

1.  **Exploración (en un Notebook de Jupyter):**
    *   Carga un archivo de `/data/raw/` en un notebook.
    *   Usa métodos como `.info()`, `.describe()`, `.head()`, `.isnull().sum()`, `.value_counts()` para "entrevistar" a tus datos y descubrir sus problemas.
    *   Prueba diferentes técnicas de limpieza en celdas separadas hasta que encuentres las que funcionan.
    *   **El objetivo aquí es DESCUBRIR, no implementar.**

2.  **Implementación (en un script `.py`):**
    *   Una vez que tienes claras las reglas de limpieza del notebook, las traduces a un script reutilizable (ej. `clean_data.py`).
    *   Este script contendrá todas las transformaciones de manera ordenada.
    *   **El objetivo aquí es CODIFICAR las reglas de forma reproducible.**

---

### 2. La Checklist de Limpieza (El "Qué" y el "Cómo")

Vamos a usar tu tabla `organizations` como ejemplo para cada punto.

#### ✅ **1. Problemas Estructurales**
Son problemas relacionados con la forma y estructura del DataFrame.

*   **Nombres de Columnas:**
    *   **Qué:** Asegurarse de que los nombres sean consistentes, sin espacios, sin caracteres especiales y en un formato estándar (como `snake_case`, todo en minúsculas y con guiones bajos).
    *   **Por qué:** Evita errores al acceder a las columnas y es una práctica estándar en bases de datos.
    *   **Cómo:**
        ```python
        df.columns = [col.lower().replace(' ', '_') for col in df.columns]
        ```

*   **Eliminar Duplicados:**
    *   **Qué:** Buscar y eliminar filas que son copias exactas de otras.
    *   **Por qué:** Los duplicados sesgan los análisis (conteos, promedios) y pueden violar restricciones de clave única en la base de datos.
    *   **Cómo:**
        ```python
        print(f"Filas antes de eliminar duplicados: {len(df)}")
        df.drop_duplicates(inplace=True)
        print(f"Filas después de eliminar duplicados: {len(df)}")
        ```

*   **Eliminar Columnas Irrelevantes:**
    *   **Qué:** Quitar columnas que no aportan valor al análisis (ej. "columna_vacia", "notas_internas").
    *   **Por qué:** Simplifica el DataFrame y reduce el consumo de memoria.
    *   **Cómo:**
        ```python
        df.drop(columns=['columna_inutil_1', 'columna_inutil_2'], inplace=True)
        ```

#### ✅ **2. Corrección de Tipos de Datos (Casting)**
Asegurarse de que cada columna tenga el tipo de dato correcto. Usa `df.info()` para diagnosticar.

*   **Columnas Numéricas:**
    *   **Qué:** Columnas que deberían ser números (enteros o decimales) pero están como texto (`object`).
    *   **Por qué:** Impide operaciones matemáticas.
    *   **Cómo:** `pd.to_numeric()` es tu mejor amigo. `errors='coerce'` convierte los valores que no se pueden transformar en `NaN` (nulos).
        ```python
        df['zip'] = pd.to_numeric(df['zip'], errors='coerce')
        df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
        ```

*   **Columnas de Fecha/Hora:** (Aunque no las tengas ahora, es fundamental saberlo)
    *   **Qué:** Fechas almacenadas como texto.
    *   **Por qué:** Impide análisis de series temporales, cálculos de duración, etc.
    *   **Cómo:**
        ```python
        # df['fecha_creacion'] = pd.to_datetime(df['fecha_creacion'], errors='coerce')
        ```

#### ✅ **3. Manejo de Valores Nulos (Missing Values)**
Decidir qué hacer con las celdas vacías (`NaN`).

*   **Estrategia:** La decisión depende del contexto.
    *   **Eliminar la fila (`.dropna()`):** Útil si la fila tiene demasiados valores nulos o si un valor nulo en una columna clave (como un ID) la hace inútil.
    *   **Rellenar el valor (`.fillna()`):** La opción más común.
        *   **Numéricas:** Rellenar con `0`, la media (`df['revenue'].mean()`) o la mediana (`df['revenue'].median()`).
        *   **Categóricas/Texto:** Rellenar con una constante como "Desconocido" o "No aplica".

*   **Cómo:**
    ```python
    # Para revenue, si un nulo significa 0 ingresos, lo rellenamos con 0.
    df['revenue'].fillna(0, inplace=True)
    
    # Para el teléfono, si es nulo, podemos poner un texto indicativo.
    df['phone'].fillna('No disponible', inplace=True)
    ```

#### ✅ **4. Limpieza de Contenido y Formato**
Aquí es donde se arregla la "suciedad" dentro de las celdas.

*   **Eliminar Espacios en Blanco:**
    *   **Qué:** Espacios al principio o al final de un texto (ej. `"  Nombre  "`).
    *   **Por qué:** Causa problemas al agrupar o unir datos. `"Madrid "` y `"Madrid"` se tratarían como dos ciudades diferentes.
    *   **Cómo:** `.str.strip()`
        ```python
        df['name'] = df['name'].str.strip()
        df['city'] = df['city'].str.strip()
        ```

*   **Estandarizar Mayúsculas/Minúsculas:**
    *   **Qué:** Tener "madrid", "Madrid" y "MADRID" en la misma columna.
    *   **Por qué:** Mismo problema que los espacios en blanco.
    *   **Cómo:** `.str.lower()`, `.str.upper()` o `.str.title()`.
        ```python
        df['state'] = df['state'].str.upper() # Para códigos de estado como 'CA', 'NY'
        df['city'] = df['city'].str.title()   # Para nombres propios como 'Madrid'
        ```

*   **Limpieza de Texto y Caracteres Especiales (Regex):**
    *   **Qué:** Eliminar caracteres no deseados, como guiones o paréntesis en números de teléfono.
    *   **Por qué:** Estandariza el formato para su uso posterior.
    *   **Cómo:** `.str.replace()` con expresiones regulares (regex).
        ```python
        # Dejar solo los dígitos en la columna de teléfono
        df['phone'] = df['phone'].str.replace(r'\D', '', regex=True)
        ```

#### ✅ **5. Validación de Datos y Outliers**
El último control de calidad.

*   **Rangos de Valores:**
    *   **Qué:** Comprobar que los valores numéricos estén dentro de un rango lógico.
    *   **Por qué:** Para detectar errores de entrada (ej. un `revenue` negativo, una `utilization` de 5000).
    *   **Cómo:** Usar `.describe()` para ver los mínimos y máximos, y luego filtrar.
        ```python
        # Asegurarse de que la utilización esté entre 0 y 100 (si es un porcentaje)
        df = df[(df['utilization'] >= 0) & (df['utilization'] <= 100)]
        ```

---

### 3. Script de Ejemplo: `clean_data.py`

Este script implementa el plan. Se colocaría en la misma carpeta que tus otros scripts (`/backend/organizations/etl/`).

```python
# /backend/organizations/etl/clean_data.py

import pandas as pd
from pathlib import Path

# --- CONFIGURACIÓN ---
INPUT_DIR = Path('../../data/raw')
OUTPUT_DIR = Path('../../data/processed')
FILE_NAME = 'organizations_raw.csv' # Asumimos que el raw tiene un nombre diferente
OUTPUT_FILE_NAME = 'organizations.csv'

def clean_organizations_data():
    """
    Lee datos crudos de organizaciones, los limpia y los guarda en la carpeta 'processed'.
    """
    # --- 1. LECTURA ---
    input_path = INPUT_DIR / FILE_NAME
    if not input_path.is_file():
        print(f"❌ ERROR: No se encontró el archivo de entrada en {input_path}")
        return

    print(f"📄 Leyendo datos desde {input_path}...")
    df = pd.read_csv(input_path, delimiter=';')
    print("Limpieza iniciada...")
    
    # --- 2. LIMPIEZA (Aplicando la checklist) ---

    # 2.1. Problemas Estructurales
    df.columns = [col.lower() for col in df.columns] # Estandarizar columnas a minúsculas
    df.drop_duplicates(inplace=True)

    # 2.2. Corrección de Tipos de Datos
    for col in ['zip', 'revenue', 'utilization']:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # 2.3. Manejo de Nulos
    df['revenue'].fillna(0, inplace=True)
    df['utilization'].fillna(0, inplace=True) # Asumimos 0 si no hay dato
    df.dropna(subset=['id', 'name'], inplace=True) # Borrar filas si el ID o el nombre son nulos

    # 2.4. Limpieza de Contenido
    text_cols = ['name', 'address', 'city', 'state', 'phone']
    for col in text_cols:
        df[col] = df[col].str.strip() # Quitar espacios
    
    df['state'] = df['state'].str.upper() # Estandarizar estado
    df['phone'] = df['phone'].str.replace(r'\D', '', regex=True) # Solo dígitos en teléfono

    # 2.5. Validación de Datos
    # Mantener solo filas con un ID de longitud válida (ej. 36 para un UUID)
    df = df[df['id'].str.len() == 36] 
    
    print("✅ Limpieza completada.")

    # --- 3. ESCRITURA ---
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True) # Asegurarse de que el directorio de salida exista
    output_path = OUTPUT_DIR / OUTPUT_FILE_NAME
    df.to_csv(output_path, index=False, sep=';')
    print(f"💾 Datos limpios guardados en {output_path}")

if __name__ == "__main__":
    clean_organizations_data()
```

**Tu flujo de trabajo final sería:**
1.  Poner el CSV sucio en `/data/raw/organizations_raw.csv`.
2.  Ejecutar `python clean_data.py`.
3.  Ejecutar `python load_supabase.py` (que ahora leerá el archivo limpio de `/data/processed/organizations.csv`).